In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
from keras import backend as K
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Dense, TimeDistributed, Lambda

In [ ]:
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
data = pd.read_csv("/content/ner_dataset.csv", encoding="latin1", on_bad_lines='skip', usecols=["Sentence #", "Word", "Tag"])
data.head(10)

,Sentence #,Word,Tag
0,Sentence: 1,Thousands,O
1,NaN,of,O
2,NaN,demonstrators,O
3,NaN,have,O
4,NaN,marched,O
5,NaN,through,O
6,NaN,London,B-geo
7,NaN,to,O
8,NaN,protest,O
9,NaN,the,O


In [ ]:
data = data.fillna(method="ffill")

In [ ]:
data.head(30)

,Sentence #,Word,Tag
0,Sentence: 1,Thousands,O
1,Sentence: 1,of,O
2,Sentence: 1,demonstrators,O
3,Sentence: 1,have,O
4,Sentence: 1,marched,O
5,Sentence: 1,through,O
6,Sentence: 1,London,B-geo
7,Sentence: 1,to,O
8,Sentence: 1,protest,O
9,Sentence: 1,the,O


In [ ]:
unique_words = set(list(data['Word'].values))
n_words = len(unique_words)
n_words

unique_tags = list(set(data["Tag"].values))
n_tags = len(unique_tags)
n_tags

17

In [ ]:
n_words

35178

In [ ]:
agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
grouped_sentence = data.groupby("Sentence #").apply(agg_func)
sentences = [s for s in grouped_sentence]

In [ ]:
sentences[0]

[('Thousands', 'O'),
 ('of', 'O'),
 ('demonstrators', 'O'),
 ('have', 'O'),
 ('marched', 'O'),
 ('through', 'O'),
 ('London', 'B-geo'),
 ('to', 'O'),
 ('protest', 'O'),
 ('the', 'O'),
 ('war', 'O'),
 ('in', 'O'),
 ('Iraq', 'B-geo'),
 ('and', 'O'),
 ('demand', 'O'),
 ('the', 'O'),
 ('withdrawal', 'O'),
 ('of', 'O'),
 ('British', 'B-gpe'),
 ('troops', 'O'),
 ('from', 'O'),
 ('that', 'O'),
 ('country', 'O'),
 ('.', 'O')]

In [ ]:
largest_sentence = max(len(sen) for sen in sentences)
print(largest_sentence)

104


In [ ]:
x = [[w[0]for w in s] for s in sentences]
X = []
for seq in x:
    new_seq = []
    for i in range(largest_sentence):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    X.append(new_seq)

In [ ]:
X

In [ ]:
from keras.preprocessing.sequence import pad_sequences

tags2index = {t:i for i,t in enumerate(unique_tags)}
y = [[tags2index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=largest_sentence, sequences=y, padding="post", value=tags2index["O"])

In [ ]:
y

array([[ 2,  2,  2, ...,  2,  2,  2],
       [12,  2,  2, ...,  2,  2,  2],
       [ 2,  2,  6, ...,  2,  2,  2],
       ...,
       [ 2,  7,  2, ...,  2,  2,  2],
       [ 2,  2,  2, ...,  2,  2,  2],
       [ 2, 14,  1, ...,  2,  2,  2]], dtype=int32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
sess = tf.Session()
K.set_session(sess)
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
batch_size = 32
def ElmoEmbedding(x):
    return elmo_model(inputs={"tokens": tf.squeeze(tf.cast(x,    tf.string)),"sequence_len": tf.constant(batch_size*[largest_sentence])
                     },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [ ]:
input_text = Input(shape=(largest_sentence,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(largest_sentence, 1024))(input_text)
x = LSTM(units=64, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2)(embedding)
x_rnn = LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2)(x)
x = add([x, x_rnn])
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
model = Model(input_text, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 104)]        0           []                               
                                                                                                  
 lambda (Lambda)                (32, None, 1024)     0           ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    (32, None, 64)       278784      ['lambda[0][0]']                 
                                                                                                  
 lstm_1 (LSTM)                  (32, None, 64)       33024       ['lstm[0][0]']                   
                                                                                              

In [ ]:
y_val = y_train[3000:4000]
x_val = X_train[3000:4000]

x_train = X_train[:3000]
y_train = y_train[:3000]

In [ ]:
y_tr = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

In [ ]:
history = model.fit(np.array(x_train), y_tr, validation_data=(np.array(x_val), y_val),batch_size=batch_size, epochs=10, verbose=1)

In [ ]:
i = 390
p = model.predict(np.array(X_test[i:i+batch_size]))[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*30)
for w, true, pred in zip(X_test[i], y_test[i], p):
    if w != "__PAD__":
        print("{:15}:{:5} ({})".format(w, unique_tags[pred], unique_tags[true]))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Word            Pred : (True)
In             :O     (O)
May            :O     (B-tim)
,              :O     (O)
U.S.           :O     (B-gpe)
officials      :O     (O)
rejected       :O     (O)
a              :O     (O)
Venezuelan     :O     (B-gpe)
request        :O     (O)
for            :O     (O)
his            :O     (O)
arrest         :O     (O)
,              :O     (O)
citing         :O     (O)
a              :O     (O)
lack           :O     (O)
of             :O     (O)
evidence       :O     (O)
against        :O     (O)
the            :O     (O)
Cuban          :O     (B-gpe)
exile          :O     (O)
who            :O     (O)
holds          :O     (O)
Venezuelan     :O     (B-gpe)
citizenship    :O     (O)
.              :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword        :O     (O)
PADword       